In [7]:
from google.colab import files
uploaded = files.upload()

Saving main_players.csv to main_players (1).csv


In [0]:
import json
import os
import csv
import collections
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
import shutil
import time
from scipy.stats import zscore
from scipy import stats
from sklearn import preprocessing
from sklearn import utils
import sklearn.feature_extraction.text as sk_text
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor

import io
import requests
from sklearn import metrics
from keras import optimizers
from keras.optimizers import Adam, sgd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
import xgboost as xgb
import lightgbm as lgb



In [0]:
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, collections.Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32, np.object):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=0, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

In [0]:
df =pd.read_csv('main_players.csv')

In [0]:
df.drop(['Unnamed: 0','Photo','Flag','Club Logo'],axis=1,inplace=True)

In [0]:
df['ShortPassing'].fillna(df['ShortPassing'].mean(), inplace = True)
df['Volleys'].fillna(df['Volleys'].mean(), inplace = True)
df['Dribbling'].fillna(df['Dribbling'].mean(), inplace = True)
df['Curve'].fillna(df['Curve'].mean(), inplace = True)
df['FKAccuracy'].fillna(df['FKAccuracy'], inplace = True)
df['LongPassing'].fillna(df['LongPassing'].mean(), inplace = True)
df['BallControl'].fillna(df['BallControl'].mean(), inplace = True)
df['HeadingAccuracy'].fillna(df['HeadingAccuracy'].mean(), inplace = True)
df['Finishing'].fillna(df['Finishing'].mean(), inplace = True)
df['Crossing'].fillna(df['Crossing'].mean(), inplace = True)
df['Weight'].fillna('200lbs', inplace = True)
df['Contract Valid Until'].fillna(2019, inplace = True)
df['Height'].fillna("5'11", inplace = True)
df['Loaned From'].fillna('None', inplace = True)
df['Joined'].fillna('Jul 1, 2018', inplace = True)
df['Jersey Number'].fillna(8, inplace = True)
df['Body Type'].fillna('Normal', inplace = True)
df['Position'].fillna('ST', inplace = True)
df['Club'].fillna('No Club', inplace = True)
df['Work Rate'].fillna('Medium/ Medium', inplace = True)
df['Skill Moves'].fillna(df['Skill Moves'].median(), inplace = True)
df['Weak Foot'].fillna(3, inplace = True)
df['Preferred Foot'].fillna('Right', inplace = True)
df['International Reputation'].fillna(1, inplace = True)
df['Wage'].fillna('€200K', inplace = True)

In [15]:
df.dropna()

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,...,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,pozicija
0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,2202,Left,5,4,4,Medium/ Medium,Messi,Yes,RF,10,1-Jul-04,None,2021,5'7,159lbs,88,88,88,92,93,93,93,92,93,93,93,91,84,84,84,91,...,47,47,47,59,84,95,70,90,86,97,93,94,87,96,91,86,91,95,95,85,68,72,59,94,48,22,94,94,75,96,33,28,26,6,11,15,14,8,€226.5M,N
1,190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,2143,Right,5,5,5,High/ Medium,Neymar,Yes,LW,10,3-Aug-17,None,2022,5'9,150lbs,84,84,84,89,89,89,89,89,89,89,89,88,81,81,81,88,...,47,47,47,60,79,87,62,84,84,96,88,87,78,95,94,90,96,94,84,80,61,81,49,82,56,36,89,87,81,94,27,24,33,9,9,15,15,11,€228.1M,N
2,177003,L. Modrić,32,Croatia,91,91,Real Madrid,€67M,€420K,2280,Right,4,4,4,High/ High,Lean,Yes,RCM,10,1-Aug-12,None,2020,5'8,146lbs,77,77,77,85,84,84,84,85,87,87,87,86,88,88,88,86,...,71,71,71,79,86,72,55,93,76,90,85,78,88,93,80,72,93,90,94,79,68,89,58,82,62,83,79,92,82,84,60,76,73,13,9,7,14,9,€137.4M,V
3,176580,L. Suárez,31,Uruguay,91,91,FC Barcelona,€80M,€455K,2346,Right,5,4,3,High/ Medium,Normal,Yes,RS,9,11-Jul-14,None,2021,6'0,190lbs,87,87,87,86,87,87,87,86,85,85,85,84,79,79,79,84,...,63,63,63,66,77,93,77,82,88,87,86,84,64,90,86,75,82,92,83,86,69,90,83,85,87,41,92,84,85,85,62,45,38,27,25,31,33,37,€164M,N
4,155862,Sergio Ramos,32,Spain,91,91,Real Madrid,€51M,€380K,2201,Right,4,3,3,High/ Medium,Normal,Yes,RCB,15,1-Aug-05,None,2020,6'0,181lbs,73,73,73,70,71,71,71,70,71,71,71,72,75,75,75,72,...,87,87,87,84,66,60,91,78,66,63,74,72,77,84,76,75,78,85,66,79,93,84,83,59,88,90,60,63,75,82,87,92,91,11,8,9,7,11,€104.6M,B
5,188545,R. Lewandowski,29,Poland,90,90,FC Bayern München,€77M,€205K,2152,Right,4,4,4,High/ Medium,Normal,Yes,ST,9,1-Jul-14,None,2021,6'0,176lbs,87,87,87,83,86,86,86,83,83,83,83,81,77,77,77,81,...,57,57,57,58,62,91,85,83,89,85,77,86,65,89,77,78,78,90,78,88,84,78,84,84,80,39,91,77,88,86,34,42,19,15,6,12,8,10,€127.1M,N
6,182493,D. Godín,32,Uruguay,90,90,Atlético Madrid,€44M,€125K,1946,Right,3,3,2,Medium/ High,Lean,Yes,CB,10,4-Aug-10,None,2019,6'2,172lbs,64,64,64,61,62,62,62,61,62,62,62,63,68,68,68,63,...,87,87,87,79,55,42,92,79,47,53,49,51,70,76,68,68,58,85,54,67,91,66,88,43,89,88,48,52,50,82,90,89,89,6,8,15,5,15,€90.2M,B
7,168542,David Silva,32,Spain,90,90,Manchester City,€60M,€285K,2115,Left,4,2,4,High/ Medium,Normal,Yes,LCM,21,14-Jul-10,None,2020,5'8,148lbs,77,77,77,85,84,84,84,85,87,87,87,85,85,85,85,85,...,57,57,57,64,84,76,54,93,82,89,82,77,87,94,70,64,92,90,90,72,64,78,52,75,57,50,89,92,75,93,59,53,29,6,15,7,6,12,€111M,V
8,215914,N. Kanté,27,France,89,90,Chelsea,€63M,€225K,2189,Right,3,3,2,Medium/ High,Lean,Yes,LDM,13,16-Jul-16,None,2023,5'6,159lbs,72,72,72,77,77,77,77,77,79,79,79,79,82,82,82,79,...,83,83,83,84,68,65,54,86,56,79,49,49,81,80,82,78,82,93,92,71,77,96,76,69,90,92,71,79,54,85,90,91,85,15,12,10,7,10,€121.3M,V
9,211110,P. Dybala,24,Argentina,89,94,Juventus,€89M,€205K,2092,Left,3,3,4,High/ Medium,Normal,Yes,LF,21,1-Jul-15,None,2022,5'10,165lbs,83,83,83,87,86,86,86,87,87,87,87,86,79,79,79,86,...,45,45,45,56,82,84,68,87,88,92,88,88,75,92,87,83,91,86,85,82,75,80,65,88,48,32,84,87,86,84,23,20,20,5,4,4,5,8,€153.5M,N


In [0]:
col = ['Age', 'Potential', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping' ,'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning' ,'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']

for x in col:
    encode_numeric_range(df,x)

In [17]:
#Label encoding

encode_text_index(df,'Preferred Foot')
encode_text_index(df,'Body Type')

array(['Akinfenwa', 'Lean', 'Messi', 'Neymar', 'Normal',
       'PLAYER_BODY_TYPE_25', 'Shaqiri', 'Stocky'], dtype=object)

In [19]:
df.columns

Index(['ID', 'Name', 'Age', 'Nationality', 'Overall', 'Potential', 'Club',
       'Value', 'Wage', 'Special', 'Preferred Foot',
       'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate',
       'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined',
       'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST',
       'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM',
       'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB',
       'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing',
       'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing',
       'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions',
       'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving',
       'GKHandling', 'GKKicking

In [0]:
df.dropna(inplace = True)

In [0]:
col = ['Age', 'Potential', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping' ,'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning' ,'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']

for x in col:
    remove_outliers(df,x,3)

In [0]:
df_x = df[['Age','Potential', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping' ,'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning' ,'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'Preferred Foot', 'Body Type']].copy()

In [23]:
df_x

,Age,Potential,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Preferred Foot,Body Type
12,0.52,0.872340,0.629630,0.670588,0.670886,0.942857,0.4250,0.790123,0.662651,0.690476,0.887324,0.887324,0.389610,0.380282,0.589041,0.876923,0.405405,0.602564,0.575758,0.850746,0.722222,0.518072,0.878049,0.939024,0.795181,0.914634,0.594937,0.918919,0.952381,0.915663,0.864198,0.129032,0.21875,0.300,0.25000,0.333333,1,1
15,0.68,0.872340,0.580247,0.270588,0.860759,0.514286,0.4375,0.518519,0.590361,0.250000,0.563380,0.450704,0.558442,0.704225,0.424658,0.800000,0.445946,0.820513,0.924242,0.537313,0.888889,0.457831,0.963415,0.951220,0.204819,0.463415,0.468354,0.837838,0.988095,1.000000,0.987654,0.064516,0.06250,0.025,0.09375,0.055556,0,4
24,0.52,0.851064,0.814815,0.917647,0.810127,0.771429,0.9500,0.777778,0.831325,0.761905,0.633803,0.802817,0.948052,0.985915,0.726027,0.876923,0.648649,0.871795,0.772727,0.701493,0.708333,0.819277,0.365854,0.463415,0.951807,0.792683,0.797468,0.864865,0.202381,0.180723,0.320988,0.161290,0.25000,0.350,0.25000,0.222222,1,1
26,0.48,0.851064,0.938272,0.894118,0.873418,0.885714,0.9375,0.876543,0.951807,0.916667,0.859155,0.845070,0.961039,0.985915,0.808219,0.846154,0.581081,1.000000,0.893939,0.686567,0.763889,0.963855,0.634146,0.597561,0.891566,0.817073,0.797468,0.864865,0.523810,0.542169,0.518519,0.451613,0.43750,0.250,0.12500,0.138889,0,1
28,0.68,0.851064,0.604938,0.329412,0.835443,0.814286,0.6625,0.567901,0.602410,0.642857,0.859155,0.774648,0.649351,0.661972,0.616438,0.800000,0.621622,0.730769,0.939394,0.671642,0.791667,0.686747,0.768293,0.963415,0.578313,0.731707,0.594937,0.797297,0.928571,0.951807,0.925926,0.258065,0.34375,0.100,0.25000,0.250000,1,4
32,0.48,0.829787,0.543210,0.941176,0.835443,0.728571,0.9625,0.827160,0.710843,0.488095,0.422535,0.816901,0.753247,0.845070,0.739726,0.876923,0.554054,0.923077,0.742424,0.761194,0.708333,0.831325,0.780488,0.365854,0.963855,0.646341,0.822785,0.797297,0.285714,0.277108,0.271605,0.161290,0.21875,0.350,0.34375,0.138889,1,4
33,0.52,0.829787,0.938272,0.741176,0.696203,0.871429,0.6250,0.790123,0.831325,0.630952,0.774648,0.830986,0.948052,0.957746,0.917808,0.815385,0.851351,0.641026,0.803030,0.925373,0.500000,0.662651,0.756098,0.902439,0.819277,0.707317,0.582278,0.770270,0.738095,0.891566,0.925926,0.387097,0.43750,0.300,0.15625,0.333333,0,4
34,0.60,0.829787,0.814815,0.894118,0.253165,0.842857,0.7500,0.925926,0.855422,0.821429,0.774648,0.901408,0.948052,0.845070,0.972603,0.892308,0.945946,0.846154,0.484848,0.686567,0.236111,0.843373,0.560976,0.317073,0.915663,0.865854,0.835443,0.797297,0.178571,0.361446,0.370370,0.193548,0.28125,0.175,0.28125,0.083333,1,4
35,0.60,0.829787,0.703704,0.541176,0.822785,0.800000,0.5250,0.679012,0.566265,0.750000,0.774648,0.718310,0.532468,0.563380,0.520548,0.830769,0.513514,0.846154,0.863636,0.686567,0.750000,0.662651,0.865854,0.963415,0.590361,0.682927,0.670886,0.824324,0.952381,0.927711,0.962963,0.161290,0.28125,0.200,0.34375,0.166667,0,4
36,0.56,0.829787,0.827160,0.811765,0.746835,0.928571,0.8625,0.864198,0.831325,0.738095,0.901408,0.873239,0.636364,0.605634,0.726027,0.892308,0.594595,0.871795,0.651515,0.820896,0.583333,0.867470,0.524390,0.756098,0.927711,0.902439,0.721519,0.837838,0.773810,0.759036,0.641975,0.225806,0.15625,0.075,0.40625,0.361111,1,4


In [0]:
df_y = df['Overall'].copy()

In [25]:
df_y.head()

12    89
15    89
24    88
26    88
28    88
Name: Overall, dtype: int64

In [26]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

chi2_best = SelectKBest(chi2, k=20)

X_ch2 = chi2_best.fit_transform(df_x,df_y)

# this is a relatively strait-forward way to get to two highest indicies
max_indx = pd.Series(np.argsort(chi2_best.scores_), name='score')\
                    .nlargest(n=20).\
                    index.values
        
print('The best features of the chi2 test are:\n')

for i in range(20):
    # the +1 factor is because 
    print('{}. {}'.format(i+1, df_x.columns[max_indx[i] + 1]))

The best features of the chi2 test are:

1. GKReflexes
2. Curve
3. Finishing
4. Potential
5. Crossing
6. ShortPassing
7. HeadingAccuracy
8. SprintSpeed
9. ShotPower
10. Balance
11. Preferred Foot
12. Acceleration
13. Aggression
14. Strength
15. Vision
16. Interceptions
17. GKKicking
18. Agility
19. LongShots
20. BallControl


In [0]:
df_sel_x = df_x[["StandingTackle", "FKAccuracy", "HeadingAccuracy", "Potential", "Finishing", "ShortPassing", "Crossing", "Agility", "Jumping", "Stamina", "GKKicking",     
                "Acceleration", "Composure", "LongShots", "Vision", "Marking", "GKPositioning", "SprintSpeed", "Reactions", "BallControl"]].copy()

In [0]:
X = np.array(df_x)

In [0]:
Y = np.array(df_y)

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1 )

In [31]:
#Linear Regression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import  linear_model

lr = linear_model.LinearRegression()
lr.fit(X_train,Y_train)
lr_pred = lr.predict(X_test)

print ('--------------------------------------------------')
print ('MODEL: Linear Regression')
print ('--------------------------------------------------')


print('Actual values:')
print(Y_test)
print('Predicted values:')
print(lr_pred)

lr_mse  = mean_squared_error(lr_pred, Y_test)
lr_rmse = np.sqrt(lr_mse)
print('RMSE Score for Linear Regression:', lr_mse)
print('R2 Score: %.2f' % r2_score(Y_test, lr_pred))

--------------------------------------------------
MODEL: Linear Regression
--------------------------------------------------
Actual values:
[69 72 76 ... 57 77 65]
Predicted values:
[69.91970131 75.42579114 76.12672033 ... 54.85544654 77.76620057
 64.06445449]
RMSE Score for Linear Regression: 3.4371507032523487
R2 Score: 0.92


In [32]:
ri = Ridge(alpha=0.1, normalize=False).fit(X_train,Y_train)
ri_pred = ri.predict(X_test)
ri_mse  = mean_squared_error(ri_pred, Y_test)
ri_rmse = np.sqrt(ri_mse)

print ('--------------------------------------------------')
print ('MODEL: Ridge')
print ('--------------------------------------------------')
print('Actual values:')
print(Y_test)
print('Predicted values:')
print(ri_pred)
print('RMSE Score for Ridge:',ri_rmse)
print('R2 for Ridge: %.2f' % r2_score(Y_test, ri_pred))

--------------------------------------------------
MODEL: Ridge
--------------------------------------------------
Actual values:
[69 72 76 ... 57 77 65]
Predicted values:
[69.91544107 75.41878208 76.12643978 ... 54.85943497 77.76044729
 64.06382002]
RMSE Score for Ridge: 1.8539469687150731
R2 for Ridge: 0.92


In [33]:
ricv = RidgeCV(cv=5).fit(X_train,Y_train)
ricv_pred = ricv.predict(X_test)
ricv_mse  = mean_squared_error(ricv_pred, Y_test)
ricv_rmse = np.sqrt(ricv_mse)

print ('--------------------------------------------------')
print ('MODEL: RidgeCV')
print ('--------------------------------------------------')
print('Actual values:')
print(Y_test)
print('Predicted values:')
print(ricv_pred)
print('RMSE Score for RidgeCV:',ricv_rmse)
print('R2 for RidgeCV: %.2f' % r2_score(Y_test, ricv_pred))

--------------------------------------------------
MODEL: RidgeCV
--------------------------------------------------
Actual values:
[69 72 76 ... 57 77 65]
Predicted values:
[69.91544107 75.41878208 76.12643978 ... 54.85943497 77.76044729
 64.06382002]
RMSE Score for RidgeCV: 1.8539469687150731
R2 for RidgeCV: 0.92


In [34]:
gdb = GradientBoostingRegressor(n_estimators=200).fit(X_train,Y_train)
gdb_pred = gdb.predict(X_test)
gdb_mse  = mean_squared_error(gdb_pred, Y_test)
gdb_rmse = np.sqrt(gdb_mse)

print ('--------------------------------------------------')
print ('MODEL: Gradient Boosting')
print ('--------------------------------------------------')
print('Actual values:')
print(Y_test)
print('Predicted values:')
print(gdb_pred)
print('RMSE Score for Gradient Boosting:',gdb_rmse)
print('R2 for Gradient Boosting: %.2f' % r2_score(Y_test, gdb_pred))

--------------------------------------------------
MODEL: Gradient Boosting
--------------------------------------------------
Actual values:
[69 72 76 ... 57 77 65]
Predicted values:
[68.16093507 71.5890343  76.81861872 ... 55.82511326 76.98275831
 65.09298557]
RMSE Score for Gradient Boosting: 1.0464339453580653
R2 for Gradient Boosting: 0.98


In [35]:
lasso = Lasso(alpha =0.0005, random_state=10).fit(X_train,Y_train)
lasso_pred = lasso.predict(X_test)
lasso_mse  = mean_squared_error(lasso_pred, Y_test)
lasso_rmse = np.sqrt(lasso_mse)

print ('--------------------------------------------------')
print ('MODEL: Lasso')
print ('--------------------------------------------------')
print('Actual values:')
print(Y_test)
print('Predicted values:')
print(lasso_pred)
print('RMSE Score for Lasso:',lasso_rmse)
print('R2 for Lasso: %.2f' % r2_score(Y_test, lasso_pred))

--------------------------------------------------
MODEL: Lasso
--------------------------------------------------
Actual values:
[69 72 76 ... 57 77 65]
Predicted values:
[69.94029423 75.39551573 76.08327905 ... 54.90991898 77.76694513
 64.01732664]
RMSE Score for Lasso: 1.8519866138180927
R2 for Lasso: 0.92


In [36]:
en = ElasticNet(alpha=0.005, l1_ratio=0.1).fit(X_train,Y_train)
en_pred = en.predict(X_test)
en_mse  = mean_squared_error(en_pred, Y_test)
en_rmse = np.sqrt(en_mse)

print ('--------------------------------------------------')
print ('MODEL: Elastic Net')
print ('--------------------------------------------------')
print('Actual values:')
print(Y_test)
print('Predicted values:')
print(en_pred)
print('RMSE Score for Elastic Net:',en_rmse)
print('R2 for Elastic Net: %.2f' % r2_score(Y_test, en_pred))

--------------------------------------------------
MODEL: Elastic Net
--------------------------------------------------
Actual values:
[69 72 76 ... 57 77 65]
Predicted values:
[69.16393756 73.85193051 75.51893219 ... 56.00077052 76.08816376
 63.78907886]
RMSE Score for Elastic Net: 2.0446546400201777
R2 for Elastic Net: 0.90


In [37]:
xgb = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.5, learning_rate = 0.1,
                max_depth = 3, alpha = 10, n_estimators = 10000).fit(X_train,Y_train)
xgb_pred = xgb.predict(X_test)
xgb_mse  = mean_squared_error(xgb_pred, Y_test)
xgb_rmse = np.sqrt(xgb_mse)

print ('--------------------------------------------------')
print ('MODEL: XGBoost')
print ('--------------------------------------------------')
print('Actual values:')
print(Y_test)
print('Predicted values:')
print(xgb_pred)
print('RMSE Score for XGBoost:',xgb_rmse)
print('R2 for XGBoost: %.2f' % r2_score(Y_test, xgb_pred))

[17:14:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
--------------------------------------------------
MODEL: XGBoost
--------------------------------------------------
Actual values:
[69 72 76 ... 57 77 65]
Predicted values:
[68.6717   71.55141  76.74771  ... 56.550507 77.16396  64.928825]
RMSE Score for XGBoost: 0.8115092136922504
R2 for XGBoost: 0.99


In [38]:
br = linear_model.BayesianRidge().fit(X_train,Y_train)
br_pred = br.predict(X_test)
br_mse  = mean_squared_error(br_pred, Y_test)
br_rmse = np.sqrt(br_mse)

print ('--------------------------------------------------')
print ('MODEL: Bayesian Ridge')
print ('--------------------------------------------------')
print('Actual values:')
print(Y_test)
print('Predicted values:')
print(br_pred)
print('RMSE Score for Bayesian Ridge:',br_rmse)
print('R2 for Bayesian Ridge: %.2f' % r2_score(Y_test, br_pred))

--------------------------------------------------
MODEL: Bayesian Ridge
--------------------------------------------------
Actual values:
[69 72 76 ... 57 77 65]
Predicted values:
[69.91414393 75.41664557 76.12634903 ... 54.86065325 77.75868892
 64.06362434]
RMSE Score for Bayesian Ridge: 1.8539463571603967
R2 for Bayesian Ridge: 0.92


In [39]:
df.head()

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,...,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,pozicija
12,189511,Sergio Busquets,0.52,Spain,89,0.872340,FC Barcelona,€51.5M,€315K,2065,1,4,3,3,Medium/ Medium,1,Yes,CDM,5,1-Sep-08,None,2023,6'2,168lbs,71,71,71,74,76,76,76,74,79,79,79,76,83,83,83,76,...,82,82,82,78,0.629630,0.670588,0.670886,0.942857,0.4250,0.790123,0.662651,0.690476,0.887324,0.887324,0.389610,0.380282,0.589041,0.876923,0.405405,0.602564,0.575758,0.850746,0.722222,0.518072,0.878049,0.939024,0.795181,0.914634,0.594937,0.918919,0.952381,0.915663,0.864198,0.129032,0.21875,0.300,0.25000,0.333333,€105.6M,V
15,138956,G. Chiellini,0.68,Italy,89,0.872340,Juventus,€27M,€215K,1841,0,4,3,2,Medium/ High,4,Yes,LCB,3,1-Jul-05,None,2020,6'2,187lbs,58,58,58,54,55,55,55,54,54,54,54,56,60,60,60,56,...,86,86,86,77,0.580247,0.270588,0.860759,0.514286,0.4375,0.518519,0.590361,0.250000,0.563380,0.450704,0.558442,0.704225,0.424658,0.800000,0.445946,0.820513,0.924242,0.537313,0.888889,0.457831,0.963415,0.951220,0.204819,0.463415,0.468354,0.837838,0.988095,1.000000,0.987654,0.064516,0.06250,0.025,0.09375,0.055556,€44.6M,B
24,188567,P. Aubameyang,0.52,Gabon,88,0.851064,Arsenal,€59M,€265K,2069,1,3,4,4,Medium/ Low,1,Yes,LM,14,31-Jan-18,None,2021,6'2,176lbs,84,84,84,83,83,83,83,83,81,81,81,81,73,73,73,81,...,52,52,52,61,0.814815,0.917647,0.810127,0.771429,0.9500,0.777778,0.831325,0.761905,0.633803,0.802817,0.948052,0.985915,0.726027,0.876923,0.648649,0.871795,0.772727,0.701493,0.708333,0.819277,0.365854,0.463415,0.951807,0.792683,0.797468,0.864865,0.202381,0.180723,0.320988,0.161290,0.25000,0.350,0.25000,0.222222,€112.1M,V
26,173731,G. Bale,0.48,Wales,88,0.851064,Real Madrid,€60M,€355K,2279,0,4,3,4,High/ Medium,1,Yes,ST,11,2-Sep-13,None,2022,6'1,181lbs,86,86,86,86,86,86,86,86,85,85,85,85,80,80,80,85,...,67,67,67,71,0.938272,0.894118,0.873418,0.885714,0.9375,0.876543,0.951807,0.916667,0.859155,0.845070,0.961039,0.985915,0.808219,0.846154,0.581081,1.000000,0.893939,0.686567,0.763889,0.963855,0.634146,0.597561,0.891566,0.817073,0.797468,0.864865,0.523810,0.542169,0.518519,0.451613,0.43750,0.250,0.12500,0.138889,€123M,N
28,164240,Thiago Silva,0.68,Brazil,88,0.851064,Paris Saint-Germain,€24M,€165K,2077,1,4,3,2,Medium/ High,4,Yes,RCB,2,1-Jul-12,None,2020,6'0,181lbs,66,66,66,66,67,67,67,66,70,70,70,69,75,75,75,69,...,84,84,84,80,0.604938,0.329412,0.835443,0.814286,0.6625,0.567901,0.602410,0.642857,0.859155,0.774648,0.649351,0.661972,0.616438,0.800000,0.621622,0.730769,0.939394,0.671642,0.791667,0.686747,0.768293,0.963415,0.578313,0.731707,0.594937,0.797297,0.928571,0.951807,0.925926,0.258065,0.34375,0.100,0.25000,0.250000,€44.4M,B
